In [49]:
import pandas as pd
import numpy as np
import requests
import json

Get the toronto housing data

In [24]:
# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:
 
url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "55076653-defc-44c8-8384-9db130337438"}
package = requests.get(url, params = params).json()
 
# Get the data by passing the resource_id to the datastore_search endpoint
# See https://docs.ckan.org/en/latest/maintaining/datastore.html for detailed parameters options
# For example, to retrieve the data content for the first resource in the datastore:
 
for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        data = requests.get(url, params = p).json()
        df_housing = pd.DataFrame(data["result"]["records"])
        break
df_housing.head(3)

,BLD_DESC,BLD_FORM,BLD_ID,BLD_TYPO,DEV_ID,DEV_NAME,FLR_ABV_GR,MRKT_UNIT,NGHBRHD_NUM,OBJECTID,POLICE_DIV,PSTL_CODE,RGI_UNITS,SCATTERED,TTL_RES_UNIT,YR_BUILT,_id,geometry
0,2 - Storey Semi-Detached House - Duplex,House Semi-Detached Duplex,4327,None,1,O'Connor Drive,2,0,43,1,54,M4A 1A4,2,NULL,2,1966,1,"{""type"": ""Point"", ""coordinates"": [-79.29980685..."
1,2 - Storey Semi-Detached House - Duplex,House Semi-Detached Duplex,4328,None,1,O'Connor Drive,2,0,43,2,54,M4A 1A4,2,NULL,2,1966,2,"{""type"": ""Point"", ""coordinates"": [-79.30096506..."
2,2 - Storey Semi-Detached House - Duplex,House Semi-Detached Duplex,4329,None,1,O'Connor Drive,2,0,43,3,54,M4A 1A4,2,NULL,2,1966,3,"{""type"": ""Point"", ""coordinates"": [-79.30030895..."


Trim the unneeded columns

In [25]:
df_housing.drop(['BLD_DESC', 'BLD_FORM', 'BLD_TYPO', 'FLR_ABV_GR', 'MRKT_UNIT', 'OBJECTID', 'POLICE_DIV', 'PSTL_CODE', 'RGI_UNITS', 'SCATTERED', 'TTL_RES_UNIT', 'YR_BUILT'], axis=1, inplace=True)
df_housing.head(10)

,BLD_ID,DEV_ID,DEV_NAME,NGHBRHD_NUM,_id,geometry
0,4327,1,O'Connor Drive,43,1,"{""type"": ""Point"", ""coordinates"": [-79.29980685..."
1,4328,1,O'Connor Drive,43,2,"{""type"": ""Point"", ""coordinates"": [-79.30096506..."
2,4329,1,O'Connor Drive,43,3,"{""type"": ""Point"", ""coordinates"": [-79.30030895..."
3,4330,1,O'Connor Drive,43,4,"{""type"": ""Point"", ""coordinates"": [-79.30116453..."
4,4331,1,O'Connor Drive,43,5,"{""type"": ""Point"", ""coordinates"": [-79.30327477..."
5,4332,1,O'Connor Drive,43,6,"{""type"": ""Point"", ""coordinates"": [-79.29955853..."
6,4333,1,O'Connor Drive,43,7,"{""type"": ""Point"", ""coordinates"": [-79.30184802..."
7,4334,1,O'Connor Drive,43,8,"{""type"": ""Point"", ""coordinates"": [-79.30005653..."
8,4335,1,O'Connor Drive,43,9,"{""type"": ""Point"", ""coordinates"": [-79.30350542..."
9,4336,1,O'Connor Drive,43,10,"{""type"": ""Point"", ""coordinates"": [-79.30137416..."


Parse the coordinates

In [52]:
df_housing['latitude'] = np.array([0 for i in range(df_housing.shape[0])])
df_housing['longitude'] = np.array([0 for i in range(df_housing.shape[0])])

for index, row in df_housing.iterrows():
    coordinate = json.loads(row['geometry'])['coordinates']
    df_housing.loc[df['_id'] == row['_id'], 'latitude'] = coordinate[1]
    df_housing.loc[df['_id'] == row['_id'], 'longitude'] = coordinate[0]

df_housing.drop(['geometry'], axis=1, inplace=True)
df_housing.head()

,BLD_ID,DEV_ID,DEV_NAME,NGHBRHD_NUM,_id,latitude,longitude
0,4327,1,O'Connor Drive,43,1,43.716338,-79.299807
1,4328,1,O'Connor Drive,43,2,43.716080,-79.300965
2,4329,1,O'Connor Drive,43,3,43.716226,-79.300309
3,4330,1,O'Connor Drive,43,4,43.716010,-79.301165
4,4331,1,O'Connor Drive,43,5,43.715724,-79.303275


In [ ]:
Model the data using DBSCAN

In [58]:
from sklearn.cluster import DBSCAN

df_housing_loc = df_housing[['latitude', 'longitude']]

db_model = DBSCAN(eps=0.01, min_samples=2).fit(df_housing_loc)
# check cluster labels generated for each row in the dataframe
db_model.labels_ 

df_housing.insert(0, 'Cluster Labels', db_model.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [60]:
max(df_housing['Cluster Labels'])

8

Grab toronto restaurant data related to the housing coordinates

In [118]:
CLIENT_ID = 'QJAIVEPO1YUXGDFTRVOGLQTL3ELR2CFUHIVLIIOYNC5HWO50' # your Foursquare ID
CLIENT_SECRET = '1NDQ0FXV1BQBXIPRB551KLSR2TXFZTW03GW05GWYPY5GQ134' # your Foursquare Secret
VERSION = '20180604'

search_query = 'Food'
radius = 3000

venues = []

for i in range (0, max(df_housing['Cluster Labels'])):
    if len(df_housing.loc[df_housing['Cluster Labels']==i]['latitude']) != 0:
        latitude = df_housing.loc[df_housing['Cluster Labels']==i]['latitude'].values[0]
        longitude = df_housing.loc[df_housing['Cluster Labels']==i]['longitude'].values[0]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
        url
        results = requests.get(url).json()
        venues.extend(results['response']['venues'])

In [134]:
venues_formatted = [[venue['categories'][0]['name'], venue['location']['lat'], venue['location']['lng']] for venue in venues if venue['categories'] != []]
df_venues = pd.DataFrame(venues_formatted)
df_venues.columns = ['name', 'latitude', 'longitude']
print(len(df_venues))
df_venues.head()

222


,name,latitude,longitude
0,Supermarket,43.730517,-79.285180
1,Deli / Bodega,43.725677,-79.318248
2,Food Court,43.721851,-79.304176
3,Food,43.724597,-79.307232
4,Convenience Store,43.699523,-79.297880


Map the housing locations in relation to local restaurants to find any possible open locations

In [137]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(max(df_housing['Cluster Labels']))
ys = [i + x + (i*x)**2 for i in range(max(df_housing['Cluster Labels']))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_housing['latitude'], df_housing['longitude'], df_housing['DEV_NAME'], df_housing['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# add markers to the map
for lat, lon, poi in zip(df_venues['latitude'], df_venues['longitude'], df_venues['name']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters